In [1]:
!nvidia-smi

Tue Nov  4 03:02:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!mkdir -p /kaggle/working/optuna_results_distilbert
!cp -r /kaggle/input/roberta-db/optuna_sst2.db /kaggle/working/optuna_results_distilbert/

In [3]:
!pip install -q evaluate
!pip install -U transformers huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you ha

In [4]:
import os
import shutil
import time
import numpy as np
import pandas as pd
import torch
import optuna

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
import evaluate

# -------- Configuration --------
MODEL_NAME = "roberta-base"   # roberta-base or "bert-base-uncased", "distilbert-base-uncased"
OUTPUT_ROOT = "optuna_results_distilbert"
N_TRIALS = 10                # number of Optuna trials to run
RANDOM_SEED = 42
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# GPU setup
n_gpus = torch.cuda.device_count()
print(f"Detected GPUs: {n_gpus}")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


2025-11-04 03:03:24.134913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762225404.368183      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762225404.436890      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Detected GPUs: 2
Using device: cuda


### load the sst-2 (Stanford Sentiment Treebank) dataset 


In [5]:
raw = load_dataset("glue", "sst2")
print(raw)
print("\nDataset splits:", raw.keys())
print("Train size:", len(raw["train"]))
print("Validation size:", len(raw["validation"]))
print("Test size:", len(raw["test"]))


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

Dataset splits: dict_keys(['train', 'validation', 'test'])
Train size: 67349
Validation size: 872
Test size: 1821


### function to fine tune and evalute a model

In [6]:
# Load metrics from Hugging Face evaluate library
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "eval_accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "eval_f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"],
        "eval_precision": precision.compute(predictions=preds, references=labels, average="binary")["precision"],
        "eval_recall": recall.compute(predictions=preds, references=labels, average="binary")["recall"],
    }

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess_function(examples, max_length=128):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length", # Note: This pads to max_length (256, 128, or 64)
        max_length=max_length,
    )

def normalize_and_tokenize(ds, max_length=128):
    def fix_labels(example):
        lab = example.get("label", 0)
        if isinstance(lab, (list, tuple)):
            lab = lab[0] if len(lab) > 0 else 0
        example["label"] = int(lab)
        return example

    ds = ds.map(fix_labels)
    tokenized = ds.map(lambda ex: preprocess_function(ex, max_length=max_length), batched=True)
    
    tokenized = tokenized.map(lambda ex: {"labels": ex["label"]}, batched=True)
    tokenized = tokenized.remove_columns(['sentence', 'label', 'idx'])
    tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    
    return tokenized

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
results_log = []
BEST_F1 = -float("inf")
BEST_MODEL_DIR = os.path.join(OUTPUT_ROOT, "best_model")

def objective(trial):
    global BEST_F1, BEST_MODEL_DIR  # ensure global tracking works
    
    # --- Hyperparameter search space ---
    model_name = MODEL_NAME
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.0, 0.2)
    max_len = trial.suggest_categorical("max_len", [64, 128, 256])

    trial_dir = os.path.join(OUTPUT_ROOT, f"trial_{trial.number}")
    os.makedirs(trial_dir, exist_ok=True)

    print(f"\nStarting trial {trial.number} (max_len={max_len})")

    # --- Tokenize dataset per trial ---
    tokenized = normalize_and_tokenize(raw, max_length=max_len)

    # --- Load model ---
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Multi-GPU support (Kaggle: 2×T4)
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    # --- Training args ---
    training_args = TrainingArguments(
        output_dir=trial_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="steps",
        logging_steps=50,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        warmup_ratio=warmup_ratio,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        save_total_limit=1,
        fp16=torch.cuda.is_available(),
        dataloader_pin_memory=True,
        report_to=[],
        remove_unused_columns=False,
        label_names=["labels"],
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # --- Train and evaluate ---
    trainer.train()
    eval_res = trainer.evaluate()
    f1_score = float(eval_res.get("eval_f1", 0.0))

    # --- Save only best model ---
    if f1_score > BEST_F1:
        print(f"New best model found (F1 = {f1_score:.4f}) at trial {trial.number}")

        # Delete previous best model folder if it exists
        if os.path.exists(BEST_MODEL_DIR):
            shutil.rmtree(BEST_MODEL_DIR)

        # Move current trial folder as new best
        shutil.move(trial_dir, BEST_MODEL_DIR)
        BEST_F1 = f1_score
    else:
        # Delete non-best trial folder to save space
        shutil.rmtree(trial_dir)

    # --- Log results safely ---
    entry = {
        "trial": trial.number,
        "accuracy": eval_res.get("eval_accuracy"),
        "f1": eval_res.get("eval_f1"),
        "precision": eval_res.get("eval_precision"),
        "recall": eval_res.get("eval_recall"),
        "loss": eval_res.get("eval_loss"),
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "num_train_epochs": num_train_epochs,
        "weight_decay": weight_decay,
        "warmup_ratio": warmup_ratio,
        "max_len": max_len,
        "model_name": model_name,
        "is_best": f1_score == BEST_F1,
    }
    results_log.append(entry)

    pd.DataFrame(results_log).to_csv(
        os.path.join(OUTPUT_ROOT, "optuna_results_manual_log.csv"),
        index=False
    )

    return f1_score


In [9]:
study_db_path = "sqlite:///" + os.path.join(OUTPUT_ROOT, "optuna_sst2.db")

study = optuna.create_study(
    direction="maximize",
    storage=study_db_path,
    study_name="sst2_hf_tuning",
    load_if_exists=True,
    
)

print(f"Starting Optuna optimization (n_trials={N_TRIALS}) ...")
study.optimize(objective, n_trials=N_TRIALS)


[I 2025-11-04 03:03:51,985] Using an existing study with name 'sst2_hf_tuning' instead of creating a new one.


Starting Optuna optimization (n_trials=10) ...

Starting trial 20 (max_len=256)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_19/2199563585.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.201900,0.203645,0.940367,0.942350,0.927948,0.957207
2,0.136400,0.202013,0.936927,0.939227,0.921909,0.957207
3,0.118900,0.199091,0.948394,0.949381,0.948315,0.950450
4,0.099600,0.208512,0.948394,0.949267,0.950339,0.948198


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


[I 2025-11-04 09:25:45,331] Trial 29 finished with value: 0.9493813273340832 and parameters: {'learning_rate': 9.57607800695512e-06, 'batch_size': 32, 'num_train_epochs': 4, 'weight_decay': 0.07668923970705205, 'warmup_ratio': 0.05626857605365691, 'max_len': 64}. Best is trial 21 with value: 0.9504504504504504.


#### Visualizing the Model Performance

In [10]:
print("\n✅ Best trial result:")
print(f"  F1-score: {study.best_trial.value:.4f}")
print(f"  Params: {study.best_trial.params}")




✅ Best trial result:
  F1-score: 0.9505
  Params: {'learning_rate': 7.750627016520314e-06, 'batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 0.07893356110753257, 'warmup_ratio': 0.11463976478740226, 'max_len': 256}


In [11]:
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice,
)

plot_optimization_history(study).write_html(os.path.join(OUTPUT_ROOT, "optimization_history.html"))
plot_param_importances(study).write_html(os.path.join(OUTPUT_ROOT, "param_importances.html"))
plot_parallel_coordinate(study).write_html(os.path.join(OUTPUT_ROOT, "parallel_coordinate.html"))
plot_slice(study).write_html(os.path.join(OUTPUT_ROOT, "slice.html"))

print("✅ Optuna visualization HTMLs saved in:", OUTPUT_ROOT)


✅ Optuna visualization HTMLs saved in: optuna_results_distilbert


In [12]:
from optuna.visualization import plot_optimization_history, plot_param_importances

# Accuracy over trials
fig1 = plot_optimization_history(study)
fig1.write_html("optuna_optimization_history.html")

# Which hyperparameters mattered most
fig2 = plot_param_importances(study)
fig2.write_html("optuna_param_importances.html")
